# 1. Data Preparation: projection interpolation

In [ ]:
from pipeline.proj import load_projection_mat, reformat_for_tigre, interpolate_projections, pad_and_reshape
# patient, scan, scan_type set by user input
odd_index, angles, prj = load_projection_mat(patient, scan, scan_type)
prj_gcbct, angles1 = reformat_for_tigre(prj, angles)
prj_ngcbct_li = interpolate_projections(prj_gcbct, odd_index)
combined = pad_and_reshape(prj_ngcbct_li, v_dim)
torch.save(combined, ...)

# 2. Aggregate projections for train/val/test

In [ ]:
from pipeline.aggregate_prj import aggregate_saved_projections
aggregate_saved_projections(data_version, 'HF', 'train')
aggregate_saved_projections(data_version, 'HF', 'validation')
aggregate_saved_projections(data_version, 'HF', 'test')

# 3. Aggregate CT volumes

In [ ]:
aggregate_ct_volumes(data_version, 'train', scan_type=0, augment=False)
aggregate_ct_volumes(data_version, 'train', scan_type=0, augment=True)
# repeat for validation/test

# 4. Training

In [ ]:
from pipeline.launcher import run_app
run_app('train_app_MK6.TrainingApp', [f'--epoch=50', f'--network=IResNet', f'--model_name={model_name}', f'--data_ver={data_version}', '--shuffle=True', '--augment==True'])

# 5. Apply model to projections and reconstruction

In [ ]:
from pipeline.apply_model import apply_model_to_projections
prj_mix = apply_model_to_projections(patient_id, scan_id, 'HF', data_version, model_name, v_dim, odd_index, angles, prj_gcbct_tensor, prj_ngcbct_li_tensor)

# 6. Inference on test scans for full 3D

In [ ]:
from pipeline.infer3d import inference_3d
vol = inference_3d(patient_id, scan_id, 'HF', data_version, model_name, 'tumor_location_panc.pt')